In [1]:
# Import our dependencies
import numpy as np
import pandas as pd
import polars as pl
import re

In [2]:
#  Import and read the input csv
wbdi_df = pd.read_csv('../../Raw_Data/World_Bank-world_development_indicators_extract-1990_2022.csv')
wbdi_df

,Country,Country Code,Series Name,Series Code,1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],...,2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022]
0,Afghanistan,AFG,Access to electricity (% of population),EG.ELC.ACCS.ZS,..,..,..,..,..,..,...,68.0408782958984,89.5,71.5,97.7,97.7,93.4308776855469,97.7,97.7,97.7,..
1,Afghanistan,AFG,Average precipitation in depth (mm per year),AG.LND.PRCP.MM,327,327,327,327,327,327,...,327,327,327,327,327,327,327,327,..,..
2,Afghanistan,AFG,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,..,..,..,..,..,..,...,8.80596447,9.52887821,10.10534763,11.81859016,12.62081718,14.20841891,14.8313194,15.53361392,16.82606226,..
3,Afghanistan,AFG,"Current health expenditure per capita, PPP (cu...",SH.XPD.CHEX.PP.CD,..,..,..,..,..,..,...,178.07901916923,202.261555308607,215.725233301209,239.188747458965,264.543988949921,299.822543673828,321.525376737099,322.525614744969,..,..
4,Afghanistan,AFG,Domestic general government health expenditure...,SH.XPD.GHED.GD.ZS,..,..,..,..,..,..,...,0.44335932,0.47265697,0.52267277,0.60012972,0.64313394,0.55239552,0.49705607,1.18621242,0.55571794,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3250,Zimbabwe,ZWE,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,59.426,58.091,56.435,54.426,52.588,50.531,...,57.458,58.846,59.591,60.306,60.709,61.414,61.292,61.124,59.253,..
3251,Zimbabwe,ZWE,Population density (people per sq. km of land ...,EN.POP.DNST,26.144223859377,26.8264572831847,27.5080806514153,27.9046607212098,28.0692619878506,28.4193899444229,...,35.0405118262893,35.8168618327517,36.5902468657102,37.3599689802249,38.1313196329327,38.9096135453018,39.6913739175391,40.5057929430012,..,..
3252,Zimbabwe,ZWE,"Population, total",SP.POP.TOTL,10113893,10377815,10641501,10794918,10858594,10994041,...,13555422,13855753,14154937,14452704,14751101,15052184,15354608,15669666,15993524,16320537
3253,Zimbabwe,ZWE,Prevalence of current tobacco use (% of adults),SH.PRV.SMOK,..,..,..,..,..,..,...,..,..,13.6,..,..,12.1,12.1,11.7,..,..


In [3]:
# Drop the extra columns
wbdi_df = wbdi_df.drop(['Country Code','Series Code'], axis=1)

In [4]:
wbdi_df

,Country,Series Name,1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],1996 [YR1996],1997 [YR1997],...,2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022]
0,Afghanistan,Access to electricity (% of population),..,..,..,..,..,..,..,..,...,68.0408782958984,89.5,71.5,97.7,97.7,93.4308776855469,97.7,97.7,97.7,..
1,Afghanistan,Average precipitation in depth (mm per year),327,327,327,327,327,327,327,327,...,327,327,327,327,327,327,327,327,..,..
2,Afghanistan,Current health expenditure (% of GDP),..,..,..,..,..,..,..,..,...,8.80596447,9.52887821,10.10534763,11.81859016,12.62081718,14.20841891,14.8313194,15.53361392,16.82606226,..
3,Afghanistan,"Current health expenditure per capita, PPP (cu...",..,..,..,..,..,..,..,..,...,178.07901916923,202.261555308607,215.725233301209,239.188747458965,264.543988949921,299.822543673828,321.525376737099,322.525614744969,..,..
4,Afghanistan,Domestic general government health expenditure...,..,..,..,..,..,..,..,..,...,0.44335932,0.47265697,0.52267277,0.60012972,0.64313394,0.55239552,0.49705607,1.18621242,0.55571794,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3250,Zimbabwe,"Life expectancy at birth, total (years)",59.426,58.091,56.435,54.426,52.588,50.531,48.955,47.993,...,57.458,58.846,59.591,60.306,60.709,61.414,61.292,61.124,59.253,..
3251,Zimbabwe,Population density (people per sq. km of land ...,26.144223859377,26.8264572831847,27.5080806514153,27.9046607212098,28.0692619878506,28.4193899444229,28.8953625436216,29.3715936409461,...,35.0405118262893,35.8168618327517,36.5902468657102,37.3599689802249,38.1313196329327,38.9096135453018,39.6913739175391,40.5057929430012,..,..
3252,Zimbabwe,"Population, total",10113893,10377815,10641501,10794918,10858594,10994041,11178171,11362401,...,13555422,13855753,14154937,14452704,14751101,15052184,15354608,15669666,15993524,16320537
3253,Zimbabwe,Prevalence of current tobacco use (% of adults),..,..,..,..,..,..,..,..,...,..,..,13.6,..,..,12.1,12.1,11.7,..,..


In [5]:
pattern = r'\[.*\]'

# Remove the bracketed year info from the column names
wbdi_df.columns=[re.sub(pattern,'',col)for col in wbdi_df.columns]
wbdi_df.columns=wbdi_df.columns.str.strip()
wbdi_df

,Country,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Afghanistan,Access to electricity (% of population),..,..,..,..,..,..,..,..,...,68.0408782958984,89.5,71.5,97.7,97.7,93.4308776855469,97.7,97.7,97.7,..
1,Afghanistan,Average precipitation in depth (mm per year),327,327,327,327,327,327,327,327,...,327,327,327,327,327,327,327,327,..,..
2,Afghanistan,Current health expenditure (% of GDP),..,..,..,..,..,..,..,..,...,8.80596447,9.52887821,10.10534763,11.81859016,12.62081718,14.20841891,14.8313194,15.53361392,16.82606226,..
3,Afghanistan,"Current health expenditure per capita, PPP (cu...",..,..,..,..,..,..,..,..,...,178.07901916923,202.261555308607,215.725233301209,239.188747458965,264.543988949921,299.822543673828,321.525376737099,322.525614744969,..,..
4,Afghanistan,Domestic general government health expenditure...,..,..,..,..,..,..,..,..,...,0.44335932,0.47265697,0.52267277,0.60012972,0.64313394,0.55239552,0.49705607,1.18621242,0.55571794,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3250,Zimbabwe,"Life expectancy at birth, total (years)",59.426,58.091,56.435,54.426,52.588,50.531,48.955,47.993,...,57.458,58.846,59.591,60.306,60.709,61.414,61.292,61.124,59.253,..
3251,Zimbabwe,Population density (people per sq. km of land ...,26.144223859377,26.8264572831847,27.5080806514153,27.9046607212098,28.0692619878506,28.4193899444229,28.8953625436216,29.3715936409461,...,35.0405118262893,35.8168618327517,36.5902468657102,37.3599689802249,38.1313196329327,38.9096135453018,39.6913739175391,40.5057929430012,..,..
3252,Zimbabwe,"Population, total",10113893,10377815,10641501,10794918,10858594,10994041,11178171,11362401,...,13555422,13855753,14154937,14452704,14751101,15052184,15354608,15669666,15993524,16320537
3253,Zimbabwe,Prevalence of current tobacco use (% of adults),..,..,..,..,..,..,..,..,...,..,..,13.6,..,..,12.1,12.1,11.7,..,..


In [6]:
# Open the list of country name corrections
corrections_df = pd.read_csv('../../Clean_Data/master_country_list/country_name_corrections.csv')

# Convert the corrections dataframe to a dictionary.
correction_dict = dict(zip(corrections_df['wrong'], corrections_df['correct']))

In [7]:
# Apply the correction dictionary to fix the known errors
wbdi_df['Country'] = wbdi_df['Country'].replace(correction_dict)

In [8]:
wbdi_df

,Country,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Afghanistan,Access to electricity (% of population),..,..,..,..,..,..,..,..,...,68.0408782958984,89.5,71.5,97.7,97.7,93.4308776855469,97.7,97.7,97.7,..
1,Afghanistan,Average precipitation in depth (mm per year),327,327,327,327,327,327,327,327,...,327,327,327,327,327,327,327,327,..,..
2,Afghanistan,Current health expenditure (% of GDP),..,..,..,..,..,..,..,..,...,8.80596447,9.52887821,10.10534763,11.81859016,12.62081718,14.20841891,14.8313194,15.53361392,16.82606226,..
3,Afghanistan,"Current health expenditure per capita, PPP (cu...",..,..,..,..,..,..,..,..,...,178.07901916923,202.261555308607,215.725233301209,239.188747458965,264.543988949921,299.822543673828,321.525376737099,322.525614744969,..,..
4,Afghanistan,Domestic general government health expenditure...,..,..,..,..,..,..,..,..,...,0.44335932,0.47265697,0.52267277,0.60012972,0.64313394,0.55239552,0.49705607,1.18621242,0.55571794,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3250,Zimbabwe,"Life expectancy at birth, total (years)",59.426,58.091,56.435,54.426,52.588,50.531,48.955,47.993,...,57.458,58.846,59.591,60.306,60.709,61.414,61.292,61.124,59.253,..
3251,Zimbabwe,Population density (people per sq. km of land ...,26.144223859377,26.8264572831847,27.5080806514153,27.9046607212098,28.0692619878506,28.4193899444229,28.8953625436216,29.3715936409461,...,35.0405118262893,35.8168618327517,36.5902468657102,37.3599689802249,38.1313196329327,38.9096135453018,39.6913739175391,40.5057929430012,..,..
3252,Zimbabwe,"Population, total",10113893,10377815,10641501,10794918,10858594,10994041,11178171,11362401,...,13555422,13855753,14154937,14452704,14751101,15052184,15354608,15669666,15993524,16320537
3253,Zimbabwe,Prevalence of current tobacco use (% of adults),..,..,..,..,..,..,..,..,...,..,..,13.6,..,..,12.1,12.1,11.7,..,..


In [9]:
# Open the master list of countries
countries_df = pd.read_csv('../../Clean_Data/master_country_list/country_profile_urls.csv')
countries_df = countries_df.drop(['profile_url'], axis=1)

In [10]:
master_countries = countries_df['country'].tolist()
master_countries

['Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Congo',
 'Democratic Republic of the Congo',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czechia',
 "Côte d'Ivoire",
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Haiti',
 'Honduras',
 'Hong Kong (China)',
 

In [11]:
# Create a list of wbdi countries
ps_countries = wbdi_df['Country'].tolist()
ps_countries

['Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andor

In [12]:
# clean the FS data based on the SS country list
no_match = []

for country in ps_countries:
    if country in master_countries:
        continue
    else:
        no_match.append(country)

In [13]:
# Likely need to do manual country name clean-up for accented characters and abbreviations.
no_match

['Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Cayman Islands',
 'Cayman Islands',
 'Cayman Islands',
 'Cayman Isla

In [14]:
wbdi_clean_df = wbdi_df.drop(wbdi_df[wbdi_df['Country'].isin(no_match)].index.tolist())
wbdi_clean_df

,Country,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
15,Albania,Access to electricity (% of population),100,100,100,100,100,100,100,100,...,99.8903121948242,99.95,99.98,99.89,99.89,100,100,100,100,..
16,Albania,Average precipitation in depth (mm per year),1485,1485,1485,1485,1485,1485,1485,1485,...,1485,1485,1485,1485,1485,1485,1485,1485,..,..
17,Albania,Current health expenditure (% of GDP),..,..,..,..,..,..,..,..,...,6.27234411,6.44164276,6.48361206,6.66606188,6.54604626,6.65958786,..,..,..,..
18,Albania,"Current health expenditure per capita, PPP (cu...",..,..,..,..,..,..,..,..,...,664.902444642237,726.53974963245,755.452047153836,803.796389477951,834.280774470738,895.594815963717,..,..,..,..
19,Albania,Domestic general government health expenditure...,..,..,..,..,..,..,..,..,...,2.57775068,2.5936265,2.731354,2.80370712,2.6750803,2.82325125,2.86730385,2.93465567,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3250,Zimbabwe,"Life expectancy at birth, total (years)",59.426,58.091,56.435,54.426,52.588,50.531,48.955,47.993,...,57.458,58.846,59.591,60.306,60.709,61.414,61.292,61.124,59.253,..
3251,Zimbabwe,Population density (people per sq. km of land ...,26.144223859377,26.8264572831847,27.5080806514153,27.9046607212098,28.0692619878506,28.4193899444229,28.8953625436216,29.3715936409461,...,35.0405118262893,35.8168618327517,36.5902468657102,37.3599689802249,38.1313196329327,38.9096135453018,39.6913739175391,40.5057929430012,..,..
3252,Zimbabwe,"Population, total",10113893,10377815,10641501,10794918,10858594,10994041,11178171,11362401,...,13555422,13855753,14154937,14452704,14751101,15052184,15354608,15669666,15993524,16320537
3253,Zimbabwe,Prevalence of current tobacco use (% of adults),..,..,..,..,..,..,..,..,...,..,..,13.6,..,..,12.1,12.1,11.7,..,..


In [15]:
# Open the list of series name update list
series_updates_df = pd.read_csv('../../Clean_Data/master_country_list/series_name_updates.csv')

# Convert the series_updates dataframe to a dictionary.
series_updates_dict = dict(zip(series_updates_df['current'], series_updates_df['updated']))

In [16]:
# Apply the series_updates dictionary to reduce verbosity
wbdi_clean_df['Series Name'] = wbdi_clean_df['Series Name'].replace(series_updates_dict)

In [17]:
# Get a list of the columns to convert to numeric
cols = wbdi_clean_df.columns.drop(['Country', 'Series Name'])

wbdi_clean_df[cols] = wbdi_clean_df[cols].apply(pd.to_numeric, errors='coerce')

### Separate the data to manage missing values

In [18]:
# Create a dictionary of the series
series_names = wbdi_clean_df['Series Name'].unique()
print(series_names)

dataframes = {}
for series_name in series_names:
    dataframes[series_name] = wbdi_clean_df[wbdi_clean_df['Series Name']==series_name]
    

['Electricity Access (% Pop)' 'Avg. Rainfall (mm/yr)' 'Current HE (% GDP)'
 "Current HE per capita (PPP Int'l $)" "Domestic general gov't HE (% GDP)"
 "Domestic general gov't HE per capita (PPP Int'l $)"
 "Domestic private HE per capita (PPP Int'l $)" 'GDP growth per capita %'
 "GDP per capita (PPP Int'l $)" 'Total Labor Force' 'LEx years'
 'Population Density (people per sq. km)' 'Total Population'
 'Tobacco use (% adults)' 'Alcohol Consumption per capita (liters)']


In [19]:
# Create a separate Polars dataframe for the series data
electricity_access_df = pl.DataFrame(dataframes['Electricity Access (% Pop)'])
electricity_access_df.head(10)

Country,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Albania""","""Electricity Ac…",100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,99.430855,99.421989,99.404579,99.385628,99.372139,99.371124,99.389366,99.428452,100.0,99.554451,99.644203,99.718643,99.9,99.890312,99.95,99.98,99.89,99.89,100.0,100.0,100.0,100.0,null
"""Algeria""","""Electricity Ac…",null,null,null,null,null,null,null,null,null,null,98.64003,98.63797,98.627357,98.615211,98.608528,98.614319,98.7,98.685249,99.3,98.82486,98.910904,99.002655,98.76466,99.187927,99.274551,99.356194,99.433693,99.530022,99.637741,99.5,99.718773,99.787827,null
"""Andorra""","""Electricity Ac…",100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,null
"""Angola""","""Electricity Ac…",null,null,null,null,null,null,null,null,null,null,24.238865,20.0,26.320255,27.355137,28.395485,29.448309,30.520388,37.5,38.49,33.846977,34.980053,34.6,37.259491,38.398178,32.0,42.0,41.785034,42.906242,45.29,45.633881,46.955444,48.223854,null
"""Antigua and Ba…","""Electricity Ac…",95.763046,95.940651,96.118134,96.294945,96.47039,96.643799,96.814484,96.981773,97.144981,97.303352,97.673126,97.745804,100.0,97.87252,97.940575,92.2,100.0,100.0,100.0,98.530594,98.691376,94.552014,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,null
"""Argentina""","""Electricity Ac…",92.1548,92.492188,92.829453,93.166046,93.501274,93.834465,94.16494,94.492012,94.815002,95.133156,95.680473,95.511063,96.096001,96.297951,96.505371,96.725266,96.964409,97.224396,97.501602,97.792206,98.82,99.034286,99.143799,99.285774,100.0,99.689034,99.899742,100.0,99.993736,100.0,100.0,100.0,null
"""Armenia""","""Electricity Ac…",null,null,null,null,null,null,null,null,null,null,98.9,100.0,98.0,99.106911,99.094437,99.8,99.113693,99.153786,99.211113,99.281815,99.8,99.494614,99.535866,99.42,99.71,100.0,99.8,99.7,99.9,100.0,100.0,100.0,null
"""Aruba""","""Electricity Ac…",100.0,99.153656,99.197128,99.239914,100.0,100.0,100.0,100.0,100.0,100.0,91.660398,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,93.356292,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,null
"""Australia""","""Electricity Ac…",100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,null


In [20]:
# Create a separate Polars dataframe for the series data
avg_rainfall_df = pl.DataFrame(dataframes['Avg. Rainfall (mm/yr)'])
avg_rainfall_df.head(10)

Country,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Albania""","""Avg. Rainfall …",1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,null,null
"""Algeria""","""Avg. Rainfall …",89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,null,null
"""Andorra""","""Avg. Rainfall …",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Angola""","""Avg. Rainfall …",1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,1010.0,null,null
"""Antigua and Ba…","""Avg. Rainfall …",1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,1030.0,null,null
"""Argentina""","""Avg. Rainfall …",591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,591.0,null,null
"""Armenia""","""Avg. Rainfall …",null,null,562.0,562.0,562.0,562.0,562.0,562.0,562.0,562.0,562.0,562.0,562.0,562.0,562.0,562.0,562.0,562.0,562.0,562.0,562.0,562.0,562.0,562.0,562.0,562.0,562.0,562.0,562.0,562.0,562.0,null,null
"""Aruba""","""Avg. Rainfall …",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Australia""","""Avg. Rainfall …",534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,534.0,null,null


In [21]:
# Create a separate Polars dataframe for the series data
current_HE_GDP_df = pl.DataFrame(dataframes['Current HE (% GDP)'])
current_HE_GDP_df.head(10)

Country,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Albania""","""Current HE (% …",null,null,null,null,null,null,null,null,null,null,5.944186,5.925833,5.6645999,6.198919,6.0557785,5.83303,5.6859889,5.708428,5.509624,5.7276206,6.238996,6.119528,6.092626,6.272344,6.441643,6.483612,6.666062,6.546046,6.659588,null,null,null,null
"""Algeria""","""Current HE (% …",null,null,null,null,null,null,null,null,null,null,3.489033,3.837877,3.7300415,3.601041,3.5440731,3.235161,3.355103,3.821418,4.201883,5.359398,5.11717,5.267481,6.00051,6.035763,6.547214,6.978492,6.607498,6.279384,6.163026,6.251709,6.32118,null,null
"""Andorra""","""Current HE (% …",null,null,null,null,null,null,null,null,null,null,5.952103,5.85758,5.99556,5.547072,5.2878232,5.594125,4.964659,4.922833,5.6700263,6.08031,7.878397,7.70434,6.74645,6.832912,6.825868,7.023242,7.008394,7.161796,7.430458,7.408406,9.051759,null,null
"""Angola""","""Current HE (% …",null,null,null,null,null,null,null,null,null,null,1.908599,4.483516,3.329461,3.547973,3.967199,2.852195,2.685535,2.974391,3.322903,3.842608,2.6951,2.645608,2.39575,2.732827,2.434129,2.605795,2.71315,2.793838,2.588796,2.655926,2.911835,null,null
"""Antigua and Ba…","""Current HE (% …",null,null,null,null,null,null,null,null,null,null,4.715426,5.438391,5.287936,5.581433,5.11668,4.990941,4.630507,4.446095,4.20027,4.177328,5.37907,5.442572,5.479215,5.509092,5.872699,5.279655,5.121902,4.874939,4.848693,4.375672,5.614729,null,null
"""Argentina""","""Current HE (% …",null,null,null,null,null,null,null,null,null,null,8.220011,8.371799,7.3933115,6.894516,7.228683,7.610789,7.640328,7.834551,8.1826973,9.455996,9.445464,9.418199,9.827267,9.780979,9.6713,10.229337,9.001898,10.442257,9.499096,9.471705,9.984381,null,null
"""Armenia""","""Current HE (% …",null,null,null,null,null,null,null,null,null,null,4.195812,4.645639,7.613588,7.179362,6.055101,5.8618803,5.850071,5.49814,6.639766,8.443777,9.235383,9.375889,9.13455,10.32345,10.178279,10.117627,9.951959,10.362705,10.005434,11.379801,12.241035,null,null
"""Aruba""","""Current HE (% …",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Australia""","""Current HE (% …",null,null,null,null,null,null,null,null,null,null,7.599617,7.682723,7.878076,7.882926,8.090034,7.97403,7.981432,8.058902,8.25374,8.55234,8.423255,8.539568,8.674817,8.750828,9.847602,10.183613,10.090231,10.125714,10.073915,10.229891,10.648995,null,null


In [22]:
# Create a separate Polars dataframe for the series data
current_HE_PPP_df = pl.DataFrame(dataframes["Current HE per capita (PPP Int'l $)"])
current_HE_PPP_df.head(10)

Country,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Albania""","""Current HE per…",null,null,null,null,null,null,null,null,null,null,222.786533,247.234911,257.917555,304.600289,324.861823,339.739179,372.026996,414.916862,452.67909,504.256292,600.657527,625.645152,643.145746,664.902445,726.53975,755.452047,803.796389,834.280774,895.594816,null,null,null,null
"""Algeria""","""Current HE per…",null,null,null,null,null,null,null,null,null,null,297.265963,339.481319,349.191809,363.143972,377.454026,371.152919,397.531769,472.991628,533.724751,684.513406,672.379745,713.429374,800.909964,790.96993,854.943519,842.430608,772.114456,741.563757,741.41943,754.209454,735.753457,null,null
"""Andorra""","""Current HE per…",null,null,null,null,null,null,null,null,null,null,1913.36792,2017.651112,2162.962953,2094.003492,2108.606273,2342.942744,2304.167198,2491.281679,2888.111677,3066.932816,3981.440906,3953.449159,3467.419948,3461.914902,3556.152537,3762.877063,3630.907139,3762.339407,4075.102821,4146.364176,4719.697773,null,null
"""Angola""","""Current HE per…",null,null,null,null,null,null,null,null,null,null,62.701258,151.784359,126.048714,136.177882,167.516712,137.857829,143.91012,179.931613,219.595564,248.715628,177.492477,177.258724,177.029478,208.895159,197.725444,189.547861,190.657022,201.4364,181.077975,171.679167,184.137077,null,null
"""Antigua and Ba…","""Current HE per…",null,null,null,null,null,null,null,null,null,null,804.226799,890.951108,877.950204,989.773132,974.711486,1031.825717,1097.620994,1166.612466,1106.798524,961.553927,1138.733895,1139.779195,1107.338983,1070.866515,1159.681192,1038.734811,1064.919304,1013.026333,1096.55068,1050.518882,1082.485519,null,null
"""Argentina""","""Current HE per…",null,null,null,null,null,null,null,null,null,null,1062.068023,1045.118602,826.50326,845.654785,878.926912,1027.996378,1136.950096,1291.940947,1417.275937,1536.959894,1693.096567,1808.443055,1920.127825,1960.465834,1896.806034,2050.680833,1824.833791,2463.523086,2216.692967,2187.828948,2089.518422,null,null
"""Armenia""","""Current HE per…",null,null,null,null,null,null,null,null,null,null,108.056657,135.51667,257.697891,284.080612,273.507648,312.866433,366.645287,405.280373,537.129643,595.041563,677.007438,738.986198,846.535084,1014.067174,1029.526857,1025.15043,1091.416808,1296.336992,1355.647719,1698.212756,1721.091937,null,null
"""Aruba""","""Current HE per…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Australia""","""Current HE per…",null,null,null,null,null,null,null,null,null,null,2152.270618,2274.253004,2428.894462,2553.167331,2748.593346,2842.850714,3022.101952,3195.534254,3310.377823,3565.958848,3608.057772,3791.200974,3807.512015,4182.513227,4689.064907,4809.133725,5059.312654,5136.068507,5337.976003,5389.842963,5929.9836,null,null


In [23]:
# Create a separate Polars dataframe for the series data
dom_gen_gov_HE_GDP_df = pl.DataFrame(dataframes["Domestic general gov't HE (% GDP)"])
dom_gen_gov_HE_GDP_df.head(10)

Country,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Albania""","""Domestic gener…",null,null,null,null,null,null,null,null,null,null,2.765835,2.753284,2.588679,2.670353,2.820827,2.665318,2.559311,2.435714,2.2712636,2.338068,2.378746,2.365557,2.455187,2.577751,2.5936265,2.731354,2.803707,2.6750803,2.823251,2.867304,2.934656,null,null
"""Algeria""","""Domestic gener…",null,null,null,null,null,null,null,null,null,null,2.510828,2.9254992,2.8079257,2.7617898,2.52882,2.241671,2.296994,2.666877,3.025163,3.798959,3.555522,3.719759,4.376141,4.288303,4.714543,4.9170022,4.472452,4.141408,4.058195,4.066515,3.956167,null,null
"""Andorra""","""Domestic gener…",null,null,null,null,null,null,null,null,null,null,3.900492,3.817918,3.976118,3.553675,3.2834096,3.595251,2.976663,2.939606,3.692734,4.121234,5.879387,5.691884,4.698115,4.803417,4.864598,5.082504,5.07096,5.177429,5.272873,5.307939,6.623335,null,null
"""Angola""","""Domestic gener…",null,null,null,null,null,null,null,null,null,null,1.110987,2.046315,1.308627,1.465597,1.686634,1.278758,1.44412,1.721988,2.1384356,2.600458,1.674096,1.713218,1.563336,1.695167,1.31816,1.234241,1.197537,1.29295,1.10391,1.094481,1.224741,null,null
"""Antigua and Ba…","""Domestic gener…",null,null,null,null,null,null,null,null,null,null,2.727691,3.212925,3.090161,3.34376,2.969373,2.776092,2.452068,2.2822845,2.132229,2.293125,3.136883,3.099451,3.084538,3.089606,3.474277,3.156583,2.860578,2.737868,2.912823,2.557132,3.381363,null,null
"""Argentina""","""Domestic gener…",null,null,null,null,null,null,null,null,null,null,4.498951,4.592103,4.033505,3.772156,3.744384,3.913399,4.018989,4.254916,4.508547,5.669828,5.568343,5.681173,6.134855,6.220423,6.3273077,6.824373,5.591187,6.599357,5.962516,6.059421,6.6165266,null,null
"""Armenia""","""Domestic gener…",null,null,null,null,null,null,null,null,null,null,0.954682,1.339086,1.171841,1.206357,1.294115,1.478411,1.586408,1.613891,1.488134,1.811691,1.686288,1.6728667,1.57817,1.375658,1.50548,1.610208,1.637343,1.372506,1.234681,1.411674,2.356828,null,null
"""Aruba""","""Domestic gener…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Australia""","""Domestic gener…",null,null,null,null,null,null,null,null,null,null,5.459285,5.463068,5.667306,5.623778,5.799511,5.728342,5.719998,5.875439,5.969924,6.24411,6.094838,6.256113,6.179476,5.9267602,6.908718,7.234067,7.174599,7.216559,7.203527,7.5498395,7.996131,null,null


In [24]:
# Create a separate Polars dataframe for the series data
dom_gen_gov_HE_PPP_df = pl.DataFrame(dataframes["Domestic general gov't HE per capita (PPP Int'l $)"])
dom_gen_gov_HE_PPP_df.head(10)

Country,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Albania""","""Domestic gener…",null,null,null,null,null,null,null,null,null,null,103.662764,114.871264,117.86635,131.214883,151.323064,155.238838,167.452437,177.039776,186.610458,205.842089,229.013045,241.848567,259.172784,273.255533,292.529819,318.249602,338.072137,340.933759,379.676537,398.656673,396.009213,null,null
"""Algeria""","""Domestic gener…",null,null,null,null,null,null,null,null,null,null,213.922798,258.776501,262.866957,278.51037,269.326657,257.175082,272.160929,330.089709,384.257367,485.210889,467.184179,503.805402,584.09943,561.970254,615.630969,593.571344,522.625134,489.079513,488.205715,490.586587,460.477862,null,null
"""Andorra""","""Domestic gener…",null,null,null,null,null,null,null,null,null,null,1253.855476,1315.086836,1434.427228,1341.501985,1309.313472,1505.770169,1381.510625,1487.636601,1880.948729,2078.767356,2971.21764,2920.765382,2414.653592,2433.665026,2534.366796,2723.078551,2627.16136,2719.882633,2891.813705,2970.766627,3453.488106,null,null
"""Angola""","""Domestic gener…",null,null,null,null,null,null,null,null,null,null,36.498135,69.275662,49.542771,56.252356,71.218851,61.807417,77.386219,104.16923,141.319476,168.316538,110.251719,114.78754,115.519792,129.577242,107.074785,89.779783,84.152683,93.222057,77.214999,70.7473,77.449551,null,null
"""Antigua and Ba…","""Domestic gener…",null,null,null,null,null,null,null,null,null,null,465.214017,526.361446,513.055991,592.95956,565.65625,573.928428,581.241199,598.849393,561.856272,527.840563,664.069193,649.084584,623.379394,600.562671,686.06494,621.035247,594.756508,568.93676,658.746214,613.920743,651.906192,null,null
"""Argentina""","""Domestic gener…",null,null,null,null,null,null,null,null,null,null,581.287872,573.268889,450.908227,462.678155,455.275106,528.586556,598.062025,701.648509,780.898369,921.563355,998.123677,1090.875031,1198.67564,1246.800108,1240.957826,1368.08588,1133.426095,1556.911445,1391.402767,1399.63999,1384.698263,null,null
"""Armenia""","""Domestic gener…",null,null,null,null,null,null,null,null,null,null,24.586367,39.062103,39.66341,47.734428,58.454897,78.907306,99.425951,118.963571,120.383856,127.671693,123.614771,131.851547,146.25528,135.130183,152.278416,163.151469,179.565058,171.695544,167.288379,210.664686,331.370495,null,null
"""Aruba""","""Domestic gener…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Australia""","""Domestic gener…",null,null,null,null,null,null,null,null,null,null,1546.11205,1617.186769,1747.290596,1821.461576,1970.386965,2042.232492,2165.829157,2329.742568,2394.393618,2603.525807,2610.692326,2777.445084,2712.268065,2832.732206,3289.676711,3416.233013,3597.394354,3660.457012,3817.011483,3977.79913,4452.713886,null,null


In [25]:
# Create a separate Polars dataframe for the series data
dom_pri_HE_PPP_df = pl.DataFrame(dataframes["Domestic private HE per capita (PPP Int'l $)"])
dom_pri_HE_PPP_df.head(10)

Country,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Albania""","""Domestic priva…",null,null,null,null,null,null,null,null,null,null,115.435286,121.4623,131.165703,144.993751,152.854839,164.578373,180.733703,212.806591,254.547993,290.027701,360.162233,378.376081,378.462993,384.546879,429.535609,431.903524,458.6215,480.192965,506.436432,null,null,null,null
"""Algeria""","""Domestic priva…",null,null,null,null,null,null,null,null,null,null,83.188034,80.547718,86.169451,84.4512,107.950709,113.68323,125.162847,142.588647,149.271722,198.949934,205.006317,209.483342,216.554383,228.842817,239.009238,248.633203,249.334093,252.269792,253.014011,263.286676,274.982009,null,null
"""Andorra""","""Domestic priva…",null,null,null,null,null,null,null,null,null,null,659.51239,702.564334,728.535656,752.501565,799.292754,837.172661,922.656493,1003.644945,1007.163069,988.165522,1010.223398,1032.683629,1052.766479,1028.249812,1021.785612,1039.798383,1003.7459,1042.456835,1183.289115,1175.597489,1266.209667,null,null
"""Angola""","""Domestic priva…",null,null,null,null,null,null,null,null,null,null,25.881059,80.337265,74.89577,78.510329,93.065873,71.862526,58.095862,70.908478,72.81079,70.87963,61.478556,57.399412,57.658536,73.957742,84.382475,94.70654,99.561288,101.940034,97.695533,95.448367,99.020896,null,null
"""Antigua and Ba…","""Domestic priva…",null,null,null,null,null,null,null,null,null,null,336.207738,363.938963,363.690384,396.813623,408.635215,456.297315,514.84703,566.098475,542.865442,433.447557,473.215247,490.08596,483.959591,467.783828,473.616245,417.699546,470.018536,443.858184,436.740189,436.109958,429.030149,null,null
"""Argentina""","""Domestic priva…",null,null,null,null,null,null,null,null,null,null,480.454832,469.751736,359.334865,380.072077,368.366574,487.247886,535.179147,587.328509,623.257773,610.972136,689.622859,707.074187,711.448439,704.923404,645.825566,676.773877,685.776881,895.925148,818.558659,782.906129,698.823049,null,null
"""Armenia""","""Domestic priva…",null,null,null,null,null,null,null,null,null,null,69.658875,72.048815,197.59662,221.491739,200.130665,197.431272,224.785516,236.801311,385.114985,440.057634,528.148128,583.858263,675.217524,857.92249,861.50193,850.897598,891.429707,1107.958161,1171.864086,1470.646267,1370.731225,null,null
"""Aruba""","""Domestic priva…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Australia""","""Domestic priva…",null,null,null,null,null,null,null,null,null,null,606.158537,657.066272,681.603806,731.705791,778.20629,800.61829,856.272806,865.791707,915.984274,962.43292,997.365389,1013.755854,1095.244003,1349.780972,1399.388107,1392.900671,1461.918367,1475.611469,1520.96436,1412.043937,1477.269634,null,null


In [26]:
# Create a separate Polars dataframe for the series data
GDP_Growth_pcp_df = pl.DataFrame(dataframes['GDP growth per capita %'])
GDP_Growth_pcp_df.head(10)

Country,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Albania""","""GDP growth per…",-11.187905,-27.566821,-6.622551,10.229949,8.969762,14.024496,9.78018,-10.361105,9.516484,13.608069,7.630022,9.314397,4.850475,5.924212,5.95657,6.067882,6.572923,6.787229,8.328108,4.05314,4.223084,2.821642,1.584873,1.187234,1.985388,2.516827,3.480293,3.898112,4.276312,2.523541,-2.745239,9.922714,6.205275
"""Algeria""","""GDP growth per…",-1.682928,-3.528168,-0.538304,-4.268935,-2.966914,1.828037,2.280472,-0.585445,3.524322,1.766867,2.35458,1.592497,4.18516,5.758042,2.842494,4.465245,0.244874,1.730369,0.664882,-0.208352,1.69217,0.964978,1.410227,0.797968,1.765941,1.646657,1.163148,-0.663171,-0.70795,-0.840857,-6.729942,1.700064,1.434777
"""Andorra""","""GDP growth per…",0.042321,-0.904014,-2.328613,-4.16503,-0.771998,-0.330384,2.661056,8.165548,2.39692,3.355457,2.836054,5.372533,0.076702,4.196837,3.882384,1.57804,4.292621,4.220375,-2.935382,-2.478037,1.222688,1.340894,-5.571256,-4.026027,2.140939,1.257416,2.574505,-1.416579,-0.00387,0.238297,-12.735078,6.459441,7.732716
"""Angola""","""GDP growth per…",-6.626409,-2.312502,-8.848294,-26.349124,-1.850621,11.278422,9.825458,3.755519,1.313926,-1.062725,-0.234946,0.838227,9.937307,-0.466061,7.127077,11.009566,7.614219,9.935109,7.163046,-2.79729,0.571221,-0.345062,4.538054,1.106331,1.030729,-2.642975,-6.011891,-3.630695,-4.67667,-4.017114,-8.672432,-1.954598,-0.096755
"""Antigua and Ba…","""GDP growth per…",3.512719,1.685237,-0.445129,3.400852,4.707692,-6.213723,4.4662,3.368633,2.777122,1.952957,4.457311,-6.000793,-0.255114,4.880932,4.606741,5.236709,11.278659,7.821253,-1.497594,-13.298868,-9.089175,-3.127481,2.258655,-1.525263,2.936153,3.011461,4.770737,2.516391,6.232667,3.787887,-17.989174,5.920502,7.829471
"""Argentina""","""GDP growth per…",-3.87739,7.59,6.450073,6.747497,4.420121,-4.116881,4.204967,6.823321,2.654348,-4.492112,-1.906987,-5.453797,-11.84595,7.71923,7.928157,7.732497,6.934982,7.916211,3.029785,-6.867955,9.844296,4.788683,-2.145284,1.265685,-3.578581,1.629664,-3.110064,1.757648,-3.60161,-2.969562,-10.812611,9.357176,4.273164
"""Armenia""","""GDP growth per…",null,-13.191147,-41.09865,-5.708269,8.012921,8.538542,6.631866,4.18898,8.322091,4.412253,7.153578,10.837976,14.224293,14.773836,11.161649,14.591456,13.976489,14.5361,7.651455,-13.545792,2.824482,5.319018,7.735371,3.76413,4.010646,3.606369,0.646136,8.024397,5.769882,8.208649,-6.704049,6.25422,13.025419
"""Aruba""","""GDP growth per…",2.195805,4.539317,2.370642,4.092615,4.800352,-0.567202,-1.830024,3.854744,-1.02709,-1.689537,4.918354,2.369193,-2.134006,0.117317,6.318777,-1.379148,-0.049084,1.838731,0.577088,-12.766043,-3.827013,2.400342,-1.833977,5.635318,-0.709567,2.885166,1.515938,4.917139,4.73833,0.181216,-18.698324,17.2253,null
"""Australia""","""GDP growth per…",2.054794,-1.644753,-0.689506,3.125986,2.984298,2.736499,2.60379,2.794518,3.575819,3.814385,2.718829,0.739613,2.817325,1.932213,3.108287,1.905683,1.363514,1.901151,1.51351,-0.207409,0.628837,0.978468,2.103808,0.828326,1.060337,0.693066,1.138424,0.604963,1.348283,0.665151,-1.278358,2.105711,2.457503


In [27]:
# Create a separate Polars dataframe for the series data
GDP_PPP_df = pl.DataFrame(dataframes["GDP per capita (PPP Int'l $)"])
GDP_PPP_df.head(10)

Country,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Albania""","""GDP per capita…",2549.746801,1909.31916,1823.503609,2057.692048,2290.143917,2666.081951,2980.418871,2717.684701,3021.179604,3471.646616,3861.333006,4300.222735,4661.402695,5000.295651,5427.767129,5865.291354,6565.633374,7284.166538,8228.354576,8813.51676,9628.640112,10207.759455,10526.250106,10570.966045,11259.303681,11658.894897,12078.799334,12770.991863,13498.184607,14407.437645,14064.038615,15532.706056,18551.716479
"""Algeria""","""GDP per capita…",6974.076379,6955.544827,7075.760231,6934.2603,6872.209302,7144.5735,7441.30154,7525.305678,7878.206579,8130.387872,8510.359279,8840.674165,9354.221725,10088.086769,10653.343033,11478.02805,11861.166056,12392.505919,12714.161655,12768.993719,13141.120258,13543.598719,13347.362795,13104.737104,13058.124574,12071.814181,11685.451418,11809.483033,12007.774713,12120.406374,11452.226624,12170.193777,13209.596769
"""Andorra""","""GDP per capita…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Angola""","""GDP per capita…",3283.170843,3315.70892,3091.201342,2330.662325,2336.379068,2654.402791,2968.586576,3133.186154,3210.082309,3220.724746,3285.953416,3388.148688,3782.892225,3839.572827,4223.637787,4835.669522,5364.437663,6056.779187,6615.114983,6471.284879,6586.465819,6700.1141,7389.31496,7643.921944,8123.048065,7274.090475,7027.146634,7216.061373,7043.95536,6882.279123,6367.437317,6523.463613,6973.696299
"""Antigua and Ba…","""GDP per capita…",11436.016972,12021.99725,12241.23488,12957.567846,13857.295377,13268.759668,14115.166242,14842.260251,15426.141723,15949.043435,17037.380875,16375.813389,16588.607513,17741.658674,19057.169487,20684.031858,23727.119214,26274.261395,26377.155754,23015.874438,21175.383309,20939.331275,20208.939288,19437.616969,19748.625384,19674.563296,20790.419172,20777.559499,22603.192865,23880.221274,19839.925589,21958.688709,25336.642599
"""Argentina""","""GDP per capita…",7183.583826,7990.188004,8699.39508,9506.505431,10138.681654,9925.129895,10531.845819,11444.474061,11880.480628,11506.698884,11542.986003,11159.329922,9990.720222,10974.324173,12162.337474,13513.670292,14896.733538,16510.448243,17336.932824,16249.73013,18063.914343,19322.227124,19641.352864,20131.680425,19683.771506,20105.198992,20307.870052,23597.117753,23294.098675,23007.836592,20787.857871,23754.36139,26504.590562
"""Armenia""","""GDP per capita…",2756.573324,2473.873548,1490.351937,1438.588508,1587.042988,1758.673557,1909.642681,2023.94715,2217.057854,2347.5021,2572.42082,2915.455786,3382.060324,3958.329986,4518.262746,5339.901702,6274.028207,7380.23154,8097.305007,7045.330725,7331.386504,7881.771735,9267.397006,9822.947441,10114.939837,10132.318975,10966.853327,12509.639606,13549.522425,14924.776928,14105.911253,15661.511345,18941.526808
"""Aruba""","""GDP per capita…",20753.527648,22429.289444,23484.268259,25024.82762,26786.145949,27192.707496,27183.85273,28718.557419,28743.509062,28656.0962,30746.652745,32184.219302,31988.303073,32657.89238,35653.537674,36264.435373,37365.055194,39080.466008,40059.857145,35169.785353,34230.32509,35780.249743,34786.593993,36932.184164,36846.848285,37343.912963,37583.840323,38865.188195,41685.363318,42510.06559,35012.391466,42887.374913,null
"""Australia""","""GDP per capita…",17381.440013,17836.174005,18254.357886,19216.566195,20170.957675,21037.224171,22130.477453,23123.426859,24377.417864,25468.218305,26521.078402,27626.05997,29016.678345,30106.290702,31746.532454,33018.163223,34822.975165,36628.968791,37517.853852,40306.642784,39363.618057,42018.117944,42847.296747,45907.75468,46882.185572,46251.

In [28]:
# Create a separate Polars dataframe for the series data
labor_force_df = pl.DataFrame(dataframes['Total Labor Force'])
labor_force_df.head(10)

Country,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Albania""","""Total Labor Fo…",1.374478e6,1.429833e6,1.438342e6,1.413557e6,1.391914e6,1.363163e6,1.343448e6,1.36059e6,1.344921e6,1.325723e6,1.317402e6,1.304704e6,1.308103e6,1.294847e6,1.281167e6,1.266465e6,1.249959e6,1.231498e6,1.212147e6,1.258916e6,1.270655e6,1.388801e6,1.328695e6,1.22794e6,1.25611e6,1.308343e6,1.354998e6,1.377319e6,1.409389e6,1.433004e6,1.347461e6,1.38474e6,1.381652e6
"""Algeria""","""Total Labor Fo…",6.367101e6,6.625804e6,6.864265e6,7.143936e6,7.424464e6,7.678597e6,7.968414e6,8.233015e6,8.477091e6,8.701391e6,8.926622e6,9.145785e6,9.355943e6,9.556009e6,9.747177e6,9.916026e6,1.0077549e7,1.0244513e7,1.0402784e7,1.0551825e7,1.0832192e7,1.1067848e7,1.1293832e7,1.17937e7,1.1279335e7,1.1709818e7,1.1918421e7,1.2067162e7,1.2221429e7,1.2396383e7,1.185782e7,1.2260834e7,1.2638622e7
"""Andorra""","""Total Labor Fo…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Angola""","""Total Labor Fo…",4.889409e6,5.044501e6,5.202054e6,5.36071e6,5.527396e6,5.7108e6,5.910932e6,6.121075e6,6.337219e6,6.557385e6,6.786985e6,7.031079e6,7.285806e6,7.554547e6,7.83839e6,8.133535e6,8.442257e6,8.762915e6,9.093722e6,9.435748e6,9.793696e6,1.017662e7,1.0567776e7,1.0968484e7,1.1377486e7,1.1795033e7,1.2232879e7,1.2689061e7,1.3153405e7,1.3629799e7,1.3979133e7,1.4470377e7,1.5042938e7
"""Antigua and Ba…","""Total Labor Fo…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Argentina""","""Total Labor Fo…",1.3417468e7,1.3851438e7,1.4265143e7,1.4693482e7,1.5067576e7,1.5197489e7,1.5564039e7,1.6000379e7,1.6317383e7,1.6415836e7,1.6591905e7,1.6648514e7,1.6474698e7,1.6943192e7,1.742515e7,1.7545905e7,1.7877291e7,1.7924078e7,1.7966575e7,1.8324357e7,1.8255867e7,1.8663922e7,1.8835226e7,1.8964106e7,1.9049721e7,1.9229476e7,1.9404277e7,1.958071e7,2.0103008e7,2.0607477e7,1.9406591e7,2.1191985e7,2.1424676e7
"""Armenia""","""Total Labor Fo…",1.621642e6,1.645616e6,1.61481e6,1.547325e6,1.500851e6,1.475016e6,1.465943e6,1.45743e6,1.450951e6,1.446058e6,1.441026e6,1.43573e6,1.430826e6,1.427991e6,1.427518e6,1.429115e6,1.432679e6,1.43808e6,1.437981e6,1.459017e6,1.459819e6,1.455297e6,1.447228e6,1.441157e6,1.433898e6,1.426241e6,1.418505e6,1.406989e6,1.395196e6,1.411206e6,1.385587e6,1.386828e6,1.386376e6
"""Aruba""","""Total Labor Fo…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Australia""","""Total Labor Fo…",8.482919e6,8.521707e6,8.589431e6,8.619628e6,8.780806e6,8.978386e6,9.10163e6,9.160933e6,9.253911e6,9.338857e6,9.523255e6,9.692915e6,9.837966e6,1.0013838e7,1.012607e7,1.0426942e7,1.0650659e7,1.093724e7,1.1228764e7,1.147185e7,1.1661196e7,1.1842239e7,1.2006301e7,1.2176891e7,1.2316359e7,1.2554514e7,1.2728416e7,1.3006679e7,1.3308365e7,1.3590354e7,1.35902e7,1.3807641e7,1.4111827e7


In [29]:
# Create a separate Polars dataframe for the series data
LEx_df = pl.DataFrame(dataframes['LEx years'])
LEx_df.head(10)

Country,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Albania""","""LEx years""",73.144,73.378,73.715,73.939,74.131,74.362,74.592,73.904,74.99,75.183,75.404,75.639,75.89,76.142,76.376,76.621,76.816,77.549,77.653,77.781,77.936,78.092,78.064,78.123,78.407,78.644,78.86,79.047,79.184,79.282,76.989,76.463,null
"""Algeria""","""LEx years""",67.416,67.688,67.757,67.719,67.361,67.454,68.749,69.171,69.451,70.032,70.478,70.823,71.23,71.287,71.762,72.061,72.334,72.602,72.941,73.62,73.808,74.123,74.202,74.615,75.11,75.622,75.732,75.743,76.066,76.474,74.453,76.377,null
"""Andorra""","""LEx years""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Angola""","""LEx years""",41.893,43.813,42.209,42.101,43.422,45.849,46.033,46.306,45.057,45.386,46.024,46.59,47.386,49.617,50.592,51.57,52.369,53.642,54.633,55.752,56.726,57.596,58.623,59.307,60.04,60.655,61.092,61.68,62.144,62.448,62.261,61.643,null
"""Antigua and Ba…","""LEx years""",73.492,73.435,73.417,73.482,73.591,73.636,73.829,73.96,74.041,74.347,74.606,75.078,75.536,75.478,75.672,75.825,75.765,76.131,76.41,76.669,76.82,77.104,77.35,77.583,77.858,77.913,78.152,78.268,78.511,78.691,78.841,78.497,null
"""Argentina""","""LEx years""",71.784,72.319,72.43,72.565,73.172,73.133,73.307,73.09,73.474,73.722,73.926,74.186,74.408,74.08,74.855,75.139,75.433,75.006,75.641,75.936,75.721,76.124,76.467,76.491,76.755,76.76,76.308,76.833,76.999,77.284,75.892,75.39,null
"""Armenia""","""LEx years""",68.821,68.643,68.627,68.841,69.073,69.311,69.783,69.91,70.514,70.257,70.624,70.932,71.018,71.436,71.421,71.792,71.987,72.327,72.398,72.813,73.16,73.305,73.454,73.676,74.058,74.436,74.664,74.906,75.064,75.439,72.173,72.043,null
"""Aruba""","""LEx years""",73.076,73.1,73.179,73.225,73.272,73.349,73.448,73.452,73.491,73.561,73.569,73.647,73.726,73.752,73.576,73.811,74.026,74.21,74.147,74.56,75.404,75.465,75.531,75.636,75.601,75.683,75.617,75.903,76.072,76.248,75.723,74.626,null
"""Australia""","""LEx years""",76.994634,77.27561,77.378049,77.878049,77.878049,77.829268,78.078049,78.480488,78.631707,78.931707,79.234146,79.634146,79.936585,80.239024,80.490244,80.841463,81.041463,81.292683,81.395122,81.543902,81.695122,81.895122,82.046341,82.14878,82.3,82.4,82.44878,82.5,82.74878,82.9,83.2,83.3,null


In [30]:
# Create a separate Polars dataframe for the series data
pop_density_df = pl.DataFrame(dataframes['Population Density (people per sq. km)'])
pop_density_df.head(10)

Country,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Albania""","""Population Den…",119.946788,119.225912,118.505073,117.784197,117.063358,116.342482,115.621642,114.900766,114.179927,113.459051,112.738212,111.685146,111.35073,110.934891,110.472226,109.908285,109.217044,108.394781,107.566204,106.843759,106.314635,106.029015,105.854051,105.660292,105.441752,105.135146,104.96719,104.870693,104.612263,104.167555,103.571131,null,null
"""Algeria""","""Population Den…",10.714047,10.97261,11.230572,11.485018,11.729662,11.956814,12.169521,12.375839,12.564204,12.74114,12.921067,13.10008,13.27798,13.459019,13.649763,13.837232,14.03809,14.268487,14.514427,14.777447,15.054684,15.343212,15.644261,15.954985,16.273887,16.602626,16.936908,17.271629,17.603512,17.930316,18.243657,null,null
"""Andorra""","""Population Den…",113.976596,117.944681,121.878723,125.86383,129.865957,133.889362,136.482979,137.621277,138.693617,139.691489,140.631915,144.297872,150.742553,157.248936,163.687234,169.842553,170.682979,166.314894,161.819149,157.131915,152.168085,150.142553,151.091489,151.844681,152.385106,152.651064,154.340426,157.1,159.602128,162.431915,165.319149,null,null
"""Angola""","""Population Den…",9.487959,9.808848,10.132756,10.458226,10.798132,11.159263,11.537138,11.928408,12.326032,12.73021,13.149966,13.589145,14.050003,14.537854,15.05665,15.601956,16.172568,16.772025,17.399151,18.053801,18.740824,19.45866,20.203972,20.97297,21.760116,22.56174,23.385535,24.230872,25.085051,25.951382,26.813577,null,null
"""Antigua and Ba…","""Population Den…",143.927273,144.622727,146.952273,149.622727,152.436364,155.45,158.631818,161.859091,164.936364,167.775,170.579545,173.215909,175.443182,177.443182,179.411364,181.520455,183.852273,186.4,189.206818,192.122727,194.761364,197.111364,199.259091,201.129545,202.809091,204.411364,205.827273,207.088636,208.240909,209.356818,210.6,null,null
"""Argentina""","""Population Den…",11.925961,12.097009,12.266017,12.433721,12.602339,12.769481,12.931447,13.087332,13.239788,13.393198,13.545843,13.695557,13.843376,13.98703,14.129768,14.276553,14.425036,14.570927,14.716233,14.866257,14.904302,15.077152,15.249543,15.42116,15.591645,15.760633,15.928135,16.094191,16.25851,16.420827,16.580893,null,null
"""Armenia""","""Population Den…",124.922339,127.068177,125.555146,121.438321,118.500632,116.711697,115.872778,114.907552,113.823323,112.610818,111.293397,110.050334,109.06347,108.328135,107.683351,107.033579,106.304391,105.528381,104.791746,104.119986,103.487636,102.879382,102.368142,101.910256,101.507903,101.109765,100.661574,100.172919,99.633193,99.072778,98.546119,null,null
"""Aruba""","""Population Den…",365.066667,377.022222,389.955556,402.0,415.055556,428.055556,441.205556,454.766667,468.638889,482.594444,495.005556,503.838889,509.894444,515.005556,519.666667,524.905556,531.144444,537.705556,544.422222,551.177778,557.45,562.711111,567.288889,571.555556,575.522222,579.205556,582.633333,585.772222,588.677778,591.344444,592.138889,null,null
"""Australia""","""Population Den…",2.221357,2.249852,2.275183,2.295512,2.317726,2.343684,2.372306,2.398115,2.422137,2.44878,2.476967,2.508975,2.537679,2.567036,2.594629,2.626407,2.662089,2.711118,2.765994,2.823588,2.867859,2.907986,2.9592,3.010574,3.055815,3.100113,3.144936,3.197366,3.245785,3.294351,3.335312,null,null


In [31]:
# Create a separate Polars dataframe for the series data
total_population_df = pl.DataFrame(dataframes['Total Population'])
total_population_df.head(10)

Country,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Albania""","""Total Populati…",3.286542e6,3.26679e6,3.247039e6,3.227287e6,3.207536e6,3.187784e6,3.168033e6,3.148281e6,3.12853e6,3.108778e6,3.089027e6,3.060173e6,3.05101e6,3.039616e6,3.026939e6,3.011487e6,2.992547e6,2.970017e6,2.947314e6,2.927519e6,2.913021e6,2.905195e6,2.900401e6,2.895092e6,2.889104e6,2.880703e6,2.876101e6,2.873457e6,2.866376e6,2.854191e6,2.837849e6,2.811666e6,2.775634e6
"""Algeria""","""Total Populati…",2.5518074e7,2.6133905e7,2.6748303e7,2.7354327e7,2.7937006e7,2.8478022e7,2.8984634e7,2.9476031e7,2.9924668e7,3.0346083e7,3.0774621e7,3.1200985e7,3.1624696e7,3.2055883e7,3.2510186e7,3.295669e7,3.343508e7,3.3983827e7,3.4569592e7,3.5196037e7,3.5856344e7,3.6543541e7,3.7260563e7,3.8000626e7,3.8760168e7,3.9543154e7,4.0339329e7,4.1136546e7,4.1927007e7,4.2705368e7,4.3451666e7,4.4177969e7,4.4903225e7
"""Andorra""","""Total Populati…",53569.0,55434.0,57283.0,59156.0,61037.0,62928.0,64147.0,64682.0,65186.0,65655.0,66097.0,67820.0,70849.0,73907.0,76933.0,79826.0,80221.0,78168.0,76055.0,73852.0,71519.0,70567.0,71013.0,71367.0,71621.0,71746.0,72540.0,73837.0,75013.0,76343.0,77700.0,79034.0,79824.0
"""Angola""","""Total Populati…",1.1828638e7,1.2228691e7,1.2632507e7,1.303827e7,1.3462031e7,1.3912253e7,1.438335e7,1.4871146e7,1.5366864e7,1.5870753e7,1.6394062e7,1.6941587e7,1.7516139e7,1.8124342e7,1.8771125e7,1.9450959e7,2.016234e7,2.0909684e7,2.1691522e7,2.2507674e7,2.3364185e7,2.4259111e7,2.5188292e7,2.6147002e7,2.7128337e7,2.8127721e7,2.9154746e7,3.0208628e7,3.1273533e7,3.2353588e7,3.3428486e7,3.4503774e7,3.5588987e7
"""Antigua and Ba…","""Total Populati…",63328.0,63634.0,64659.0,65834.0,67072.0,68398.0,69798.0,71218.0,72572.0,73821.0,75055.0,76215.0,77195.0,78075.0,78941.0,79869.0,80895.0,82016.0,83251.0,84534.0,85695.0,86729.0,87674.0,88497.0,89236.0,89941.0,90564.0,91119.0,91626.0,92117.0,92664.0,93219.0,93763.0
"""Argentina""","""Total Populati…",3.2637657e7,3.3105763e7,3.3568285e7,3.402724e7,3.4488696e7,3.494611e7,3.5389362e7,3.5815971e7,3.6233195e7,3.6653031e7,3.7070774e7,3.7480493e7,3.7885028e7,3.8278164e7,3.8668796e7,3.9070501e7,3.9476851e7,3.9876111e7,4.0273769e7,4.0684338e7,4.0788453e7,4.126149e7,4.1733271e7,4.2202935e7,4.26695e7,4.3131966e7,4.3590368e7,4.4044811e7,4.4494502e7,4.4938712e7,4.5376763e7,4.5808747e7,4.623483e7
"""Armenia""","""Total Populati…",3.556539e6,3.617631e6,3.574555e6,3.457349e6,3.373713e6,3.322782e6,3.298898e6,3.271418e6,3.24055e6,3.20603e6,3.168523e6,3.133133e6,3.105037e6,3.084102e6,3.065745e6,3.047246e6,3.026486e6,3.004393e6,2.983421e6,2.964296e6,2.946293e6,2.928976e6,2.914421e6,2.901385e6,2.88993e6,2.878595e6,2.865835e6,2.851923e6,2.836557e6,2.820602e6,2.805608e6,2.790974e6,2.780469e6
"""Aruba""","""Total Populati…",65712.0,67864.0,70192.0,72360.0,74710.0,77050.0,79417.0,81858.0,84355.0,86867.0,89101.0,90691.0,91781.0,92701.0,93540.0,94483.0,95606.0,96787.0,97996.0,99212.0,100341.0,101288.0,102112.0,102880.0,103594.0,104257.0,104874.0,105439.0,105962.0,106442.0,106585.0,106537.0,106445.0
"""Australia""","""Total Populati…",1.7065128e7,1.7284036e7,1.7478635e7,1.7634808e7,1.7805468e7,1.8004882e7,1.8224767e7,1.8423037e7,1.8607584e7,1.8812264e7,1.9028802e7,1.9274701e7,1.949521e7,1.9720737e7,1.9932722e7,2.0176844e7,2.0450966e7,2.0827622e7,2.1249199e7,2.1691653e7,2.203175e7,2.2340024e7,2.2733465e7,2.3128129e7,2.3475686e7,2.3815995e7,2.4190907e7,2.4594202e7,2.4966643e7,2.5340217e7,2.5655289e7,2.5688079e7,2.5978935e7


In [32]:
# Create a separate Polars dataframe for the series data
tobacco_df = pl.DataFrame(dataframes['Tobacco use (% adults)'])
tobacco_df.head(10)

Country,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Albania""","""Tobacco use (%…",null,null,null,null,null,null,null,null,null,null,35.0,null,null,null,null,31.3,null,null,null,null,28.0,null,null,null,null,25.1,null,null,23.0,23.0,22.4,null,null
"""Algeria""","""Tobacco use (%…",null,null,null,null,null,null,null,null,null,null,22.2,null,null,null,null,21.8,null,null,null,null,21.5,null,null,null,null,21.1,null,null,21.0,21.0,21.0,null,null
"""Andorra""","""Tobacco use (%…",null,null,null,null,null,null,null,null,null,null,35.9,null,null,null,null,34.8,null,null,null,null,34.0,null,null,null,null,33.0,null,null,32.0,32.0,31.8,null,null
"""Angola""","""Tobacco use (%…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Antigua and Ba…","""Tobacco use (%…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Argentina""","""Tobacco use (%…",null,null,null,null,null,null,null,null,null,null,34.0,null,null,null,null,31.5,null,null,null,null,28.9,null,null,null,null,26.6,null,null,24.9,24.9,24.5,null,null
"""Armenia""","""Tobacco use (%…",null,null,null,null,null,null,null,null,null,null,32.0,null,null,null,null,30.3,null,null,null,null,28.6,null,null,null,null,26.9,null,null,25.8,25.8,25.5,null,null
"""Aruba""","""Tobacco use (%…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Australia""","""Tobacco use (%…",null,null,null,null,null,null,null,null,null,null,24.4,null,null,null,null,21.1,null,null,null,null,18.2,null,null,null,null,15.7,null,null,14.0,14.0,13.6,null,null


In [33]:
# Create a separate Polars dataframe for the series data
alcohol_df = pl.DataFrame(dataframes['Alcohol Consumption per capita (liters)'])
alcohol_df.head(10)

Country,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Albania""","""Alcohol Consum…",null,null,null,null,null,null,null,null,null,null,6.57,null,null,null,null,7.65,null,null,null,null,7.69,null,null,null,null,6.74,null,null,7.17,null,null,null,null
"""Algeria""","""Alcohol Consum…",null,null,null,null,null,null,null,null,null,null,0.58,null,null,null,null,0.81,null,null,null,null,0.65,null,null,null,null,0.93,null,null,0.95,null,null,null,null
"""Andorra""","""Alcohol Consum…",null,null,null,null,null,null,null,null,null,null,13.28,null,null,null,null,12.55,null,null,null,null,11.11,null,null,null,null,11.01,null,null,11.02,null,null,null,null
"""Angola""","""Alcohol Consum…",null,null,null,null,null,null,null,null,null,null,2.76,null,null,null,null,4.89,null,null,null,null,8.16,null,null,null,null,7.96,null,null,6.94,null,null,null,null
"""Antigua and Ba…","""Alcohol Consum…",null,null,null,null,null,null,null,null,null,null,5.13,null,null,null,null,5.37,null,null,null,null,5.6,null,null,null,null,5.89,null,null,6.38,null,null,null,null
"""Argentina""","""Alcohol Consum…",null,null,null,null,null,null,null,null,null,null,8.75,null,null,null,null,8.66,null,null,null,null,9.09,null,null,null,null,9.55,null,null,9.65,null,null,null,null
"""Armenia""","""Alcohol Consum…",null,null,null,null,null,null,null,null,null,null,4.23,null,null,null,null,5.79,null,null,null,null,5.69,null,null,null,null,5.84,null,null,5.55,null,null,null,null
"""Aruba""","""Alcohol Consum…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Australia""","""Alcohol Consum…",null,null,null,null,null,null,null,null,null,null,11.72,null,null,null,null,12.08,null,null,null,null,12.39,null,null,null,null,10.83,null,null,10.51,null,null,null,null


In [34]:
# Work with the missing values
alcohol_df = alcohol_df.fill_none(value=0)

alcohol_df

AttributeError: 'DataFrame' object has no attribute 'fill_none'

In [ ]:
wbdi_clean_df.to_csv('../../Clean_Data/Clean_CSV_Files/world_bank_dev_indicators.csv', index=False)

In [ ]:
#fsle_clean_transform_df = fsle_clean_df.set_index('Entity').T
#fsle_clean_transform_df

In [ ]:
#fsle_clean_transform_df.to_csv('food_supply_life_expectancy_transform.csv')